# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [16]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import BernoulliNB


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Noemi
[nltk_data]     Ramiro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Noemi
[nltk_data]     Ramiro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Noemi
[nltk_data]     Ramiro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql('select * from DisasterResponse', con=engine)
    X = df.iloc[:,1]
    Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Tokenize into words
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    words = [x for x in tokens if x not in stopwords.words("english")]
    # Reduce words to their stems
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(w) for w in words]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return lemmatized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))),
                    ])

In [5]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split into train and test sample
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.33, random_state=13)

# train classifier
pipeline.fit(Xtrain, ytrain)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
ypred = pipeline.predict(Xtest)

print(classification_report(ytest, ypred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.70      0.28      0.40        58
               request       0.87      0.42      0.56      1449
                 offer       0.00      0.00      0.00        36
           aid_related       0.79      0.60      0.68      3572
          medical_help       0.69      0.05      0.09       696
      medical_products       0.63      0.05      0.08       421
     search_and_rescue       0.60      0.02      0.05       244
              security       0.00      0.00      0.00       149
              military       0.70      0.07      0.13       270
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.22      0.35       533
                  food       0.88      0.38      0.53       977
               shelter       0.81      0.25      0.38       752
              clothing       0.72      0.11      0.18       123
                 money       0.86      

C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-p

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0)
}
cv = GridSearchCV(pipeline, param_grid=parameters)
print(cv)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.33, random_state=13)
cv.fit(Xtrain, ytrain)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [10]:
ypred_cv = cv.predict(Xtest)
print(classification_report(ytest, ypred_cv, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.47      0.28      0.35        58
               request       0.88      0.45      0.59      1449
                 offer       0.00      0.00      0.00        36
           aid_related       0.79      0.57      0.66      3572
          medical_help       0.72      0.04      0.08       696
      medical_products       0.73      0.07      0.13       421
     search_and_rescue       0.79      0.05      0.09       244
              security       0.00      0.00      0.00       149
              military       0.50      0.03      0.06       270
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.29      0.44       533
                  food       0.88      0.45      0.60       977
               shelter       0.85      0.26      0.40       752
              clothing       1.00      0.03      0.06       123
                 money       0.78      

C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-p

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(KNeighborsClassifier()))])

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0)
}

knn_clf = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
knn_clf = knn_clf.fit(Xtrain, ytrain)

knn_ypred = knn_clf.predict(Xtest)

print(classification_report(ytest, knn_ypred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.25      0.31      0.28        58
               request       0.77      0.04      0.08      1449
                 offer       0.00      0.00      0.00        36
           aid_related       0.77      0.02      0.04      3572
          medical_help       0.00      0.00      0.00       696
      medical_products       0.42      0.01      0.02       421
     search_and_rescue       0.00      0.00      0.00       244
              security       0.00      0.00      0.00       149
              military       0.00      0.00      0.00       270
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.03      0.06       533
                  food       0.80      0.04      0.08       977
               shelter       0.62      0.03      0.05       752
              clothing       0.50      0.02      0.03       123
                 money       0.75      

C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-p

In [17]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(BernoulliNB()))])

parameters = {'vect__max_df': (0.5, 0.75, 1.0),
            'vect__ngram_range': ((1, 1), (1,2)),
            'vect__max_features': (None, 5000,10000),
            'tfidf__use_idf': (True, False)}

bernoulli_clf = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
bernoulli_clf = bernoulli_clf.fit(Xtrain, ytrain)

bernoulli_ypred = bernoulli_clf.predict(Xtest)

print(classification_report(ytest, bernoulli_ypred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.00      0.00      0.00        58
               request       0.91      0.07      0.12      1449
                 offer       0.00      0.00      0.00        36
           aid_related       0.82      0.37      0.51      3572
          medical_help       0.70      0.03      0.06       696
      medical_products       0.54      0.04      0.07       421
     search_and_rescue       0.16      0.02      0.03       244
              security       0.10      0.01      0.02       149
              military       0.23      0.02      0.04       270
           child_alone       0.00      0.00      0.00         0
                 water       0.54      0.03      0.05       533
                  food       0.53      0.02      0.04       977
               shelter       0.42      0.02      0.04       752
              clothing       0.11      0.02      0.03       123
                 money       0.17      

C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Noemi Ramiro\Anaconda3\lib\site-p

### 9. Export your model as a pickle file

In [15]:
joblib.dump(cv.best_estimator_, 'random_forest_best.pkl')

['random_forest_best.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.